In [1]:
pip install xgboost

Looking in indexes: https://mirrors.aliyun.com/pypi/simple
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install gensim

Looking in indexes: https://mirrors.aliyun.com/pypi/simple
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install python-Levenshtein

Looking in indexes: https://mirrors.aliyun.com/pypi/simple
     |████████████████████████████████| 50 kB 2.4 MB/s eta 0:00:011
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=171676 sha256=c03b8fb3f8567e68f66aa6a9af4bd2dbf5f5f1ed27c06ab30b6f9959351ff3f2
  Stored in directory: /home/featurize/.cache/pip/wheels/5f/35/e3/c8f279a73d74d36c4253da6d12b8ee33ba8443bb0cab7d0936
Successfully built python-Levenshtein
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import time
TRAIN_PATH = "dataset/yelp_review_small/train.csv"
TEST_PATH = "dataset/yelp_review_small/test.csv"
df_train = pd.read_csv(TRAIN_PATH, header=None)
df_test = pd.read_csv(TEST_PATH, header=None)

# Bayes

In [2]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(token_pattern='\[?\w+\]?')
X_train = vectorizer.fit_transform(df_train[1])
y_train = df_train[0]

In [3]:
X_test = vectorizer.transform(df_test[1])
y_test = df_test[0]

In [4]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [5]:
# Predicting results with the model on the test set
y_pred = clf.predict(X_test)

In [6]:
# Test set effect test
from sklearn import metrics

print(metrics.classification_report(y_test, y_pred))
print("accuracy:", metrics.accuracy_score(y_test, y_pred))

              precision    recall  f1-score   support

           1       0.58      0.28      0.38        74
           2       0.29      0.09      0.13        93
           3       0.17      0.05      0.08       146
           4       0.43      0.70      0.53       353
           5       0.61      0.57      0.59       334

    accuracy                           0.48      1000
   macro avg       0.42      0.34      0.34      1000
weighted avg       0.45      0.47      0.44      1000

accuracy: 0.475


# SVM

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(token_pattern='\[?\w+\]?')
X_train = vectorizer.fit_transform(df_train[1])
y_train = df_train[0]

In [8]:
X_test = vectorizer.transform(df_test[1])
y_test = df_test[0]

In [9]:
from sklearn import svm
t = time.perf_counter()
clf = svm.SVC(gamma='scale')
clf.fit(X_train, y_train)
t = time.perf_counter()-t
print(t)

87.27177980728447


In [10]:
# Predict results with the model on the test set
y_pred = clf.predict(X_test)

In [11]:
# Test set effect test
from sklearn import metrics
print(metrics.classification_report(y_test, y_pred))
print("accuracy:", metrics.accuracy_score(y_test, y_pred))

              precision    recall  f1-score   support

           1       0.66      0.31      0.42        74
           2       0.40      0.18      0.25        93
           3       0.45      0.14      0.21       146
           4       0.46      0.66      0.54       353
           5       0.62      0.68      0.65       334

    accuracy                           0.52      1000
   macro avg       0.52      0.39      0.41      1000
weighted avg       0.52      0.52      0.49      1000

accuracy: 0.52


# XGboost

In [12]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(token_pattern='\[?\w+\]?', max_features=2000)
X_train = vectorizer.fit_transform(df_train[1])
y_train = df_train[0]

In [13]:
X_test = vectorizer.transform(df_test[1])
y_test = df_test[0]

In [14]:
import xgboost as xgb
param = {
    'booster':'gbtree',
    'max_depth': 6, 
    'colsample_bytree': 0.8,
    'objective': 'multi:softmax',
    'eval_metric': 'error',
    'eta': 0.3,
    'nthread': 10,
    'num_class': 6,
}
dmatrix = xgb.DMatrix(X_train, label=y_train)
model = xgb.train(param, dmatrix, num_boost_round=200)

In [15]:
# Predict results with the model on the test set
dmatrix = xgb.DMatrix(X_test)
y_pred = model.predict(dmatrix)

In [16]:
# Test set effect test
from sklearn import metrics
print(metrics.classification_report(y_test, y_pred))
print("accuracy:", metrics.accuracy_score(y_test, y_pred))

              precision    recall  f1-score   support

           1       0.57      0.41      0.47        74
           2       0.43      0.22      0.29        93
           3       0.41      0.25      0.31       146
           4       0.47      0.59      0.52       353
           5       0.58      0.65      0.61       334

    accuracy                           0.51      1000
   macro avg       0.49      0.42      0.44      1000
weighted avg       0.50      0.51      0.50      1000

accuracy: 0.51


# LSTM

In [17]:
# Input format required by word2vec: list(word)
wv_input = df_train[1].map(lambda s: s.split(" "))
wv_input.head()

0    [my, wife, took, me, here, on, my, birthday, f...
1    [i, have, no, idea, why, some, people, give, b...
2    [love, the, gyro, plate, ., rice, is, so, good...
3    [rosie, ,, dakota, ,, and, i, love, chaparral,...
4    [general, manager, scott, petello, is, a, good...
Name: 1, dtype: object

In [18]:
from gensim import models
# Word2Vec 
word2vec = models.Word2Vec(wv_input, 
                           vector_size=64,   # Word vector dimension
                           min_count=1,      # Minimum word frequency, because of the small amount of data, here card 1
                           epochs=10)      # Ieration rounds

In [19]:
import torch
from torch import nn
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence,pad_packed_sequence
from torch.utils.data import Dataset, DataLoader
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [20]:
# Hyperparameters
learning_rate = 5e-4
input_size = 768
num_epoches = 1
batch_size = 100
embed_size = 64
hidden_size = 64
num_layers = 2

In [21]:
# Datasets
class MyDataset(Dataset):
    def __init__(self, df):
        self.data = []
        self.label = df[0].tolist()
        for s in df[1].tolist():
            vectors = []
            for w in s.split(" "):
                if w in word2vec.wv.key_to_index:
                    vectors.append(word2vec.wv[w])
            vectors = torch.Tensor(vectors)
            self.data.append(vectors)
    
    def __getitem__(self, index):
        data = self.data[index]
        label = self.label[index]
        return data, label

    def __len__(self):
        return len(self.label)

def collate_fn(data):
    data.sort(key=lambda x: len(x[0]), reverse=True) 
    data_length = [len(sq[0]) for sq in data]
    x = [i[0] for i in data]
    y = [i[1] for i in data]
    data = pad_sequence(x, batch_first=True, padding_value=0)   # Necessary operations for processing variable-length sequences with RNN
    return data, torch.tensor(y, dtype=torch.float32), data_length


# Training set
train_data = MyDataset(df_train)
train_loader = DataLoader(train_data, batch_size=batch_size, collate_fn=collate_fn, shuffle=True)

# Test sets
test_data = MyDataset(df_test)
test_loader = DataLoader(test_data, batch_size=batch_size, collate_fn=collate_fn, shuffle=True)

In [83]:
# Network structure
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_size * 2, 5)  # Bidirectional, output dimension to be *2
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x, lengths):
        h0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_size).to(device)  # Two-way, first dimension to be *2
        c0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_size).to(device)
        
        packed_input = torch.nn.utils.rnn.pack_padded_sequence(input=x, lengths=lengths, batch_first=True)
        packed_out, (h_n, h_c) = self.lstm(packed_input, (h0, c0))

        lstm_out = torch.cat([h_n[-2], h_n[-1]], 1)  # Bidirectional, so the last two dimensions are stitched together to give the output of the last time step
        out = self.fc(lstm_out)
        out = self.softmax(out)
        return out

lstm = LSTM(embed_size, hidden_size, num_layers)
lstm.cuda()

LSTM(
  (lstm): LSTM(64, 64, num_layers=2, batch_first=True, bidirectional=True)
  (fc): Linear(in_features=128, out_features=5, bias=True)
  (softmax): Softmax(dim=1)
)

In [180]:
from sklearn import metrics
# In test set effect test
def test():
    y_pred, y_true = [], []

    with torch.no_grad():
        for x, labels, lengths in test_loader:
            x = x.to(device)
            outputs = lstm(x, lengths)          # Forward propagation
            #outputs = outputs.view(-1)          # Flattening the output
            outputs = outputs.cpu()
            y_pred.append(outputs)
            y_true.append(labels)

    y_prob = torch.cat(y_pred)
    y_true = torch.cat(y_true)
    y_pred = y_prob.clone()
    
    print(y_prob,torch.max(y_prob,dim=1))

    #print(metrics.classification_report(y_true, y_pred))
    #print("accuracy:", metrics.accuracy_score(y_true, y_pred))

In [181]:
test()

tensor([[0.2068, 0.1927, 0.1979, 0.1980, 0.2046],
        [0.2055, 0.1863, 0.2046, 0.2019, 0.2017],
        [0.1984, 0.1851, 0.2051, 0.2008, 0.2107],
        ...,
        [0.2113, 0.1867, 0.1987, 0.1966, 0.2067],
        [0.2052, 0.1995, 0.2046, 0.1911, 0.1995],
        [0.2140, 0.1902, 0.2075, 0.1970, 0.1913]]) torch.return_types.max(
values=tensor([0.2068, 0.2055, 0.2107, 0.2121, 0.2074, 0.2081, 0.2142, 0.2135, 0.2121,
        0.2110, 0.2071, 0.2056, 0.2097, 0.2077, 0.2085, 0.2182, 0.2076, 0.2079,
        0.2096, 0.2069, 0.2081, 0.2093, 0.2128, 0.2074, 0.2111, 0.2070, 0.2110,
        0.2153, 0.2115, 0.2076, 0.2068, 0.2062, 0.2131, 0.2089, 0.2100, 0.2139,
        0.2102, 0.2190, 0.2045, 0.2119, 0.2102, 0.2120, 0.2083, 0.2087, 0.2126,
        0.2129, 0.2065, 0.2092, 0.2135, 0.2080, 0.2058, 0.2181, 0.2147, 0.2180,
        0.2100, 0.2149, 0.2157, 0.2110, 0.2081, 0.2070, 0.2060, 0.2131, 0.2136,
        0.2116, 0.2087, 0.2061, 0.2096, 0.2085, 0.2068, 0.2125, 0.2088, 0.2069,
        0.2155,

In [63]:
# Define loss functions and optimisers
criterion = CrossEntropyLoss()
optimizer = torch.optim.Adam(lstm.parameters(), lr=learning_rate)

In [64]:
# Iterative training
t = time.perf_counter()
for epoch in range(num_epoches):
    total_loss = 0
    for i, (x, labels, lengths) in enumerate(train_loader):
        x = x.to(device)
        labels = labels.to(device)
        outputs = lstm(x, lengths)          # Forward propagation
        logits = outputs.view(-1)           # Flattening the output
        loss = criterion(logits, labels)    # loss calculation
        total_loss += loss
        optimizer.zero_grad()               # Gradient clearing
        loss.backward(retain_graph=True)    # Back propagation, calculating gradients
        optimizer.step()                    # Gradient update
        if (i+1) % 10 == 0:                                                                                                                                                            
            print("epoch:{}, step:{}, loss:{}".format(epoch+1, i+1, total_loss/10))
            total_loss = 0
    
    # test
    test()

ValueError: Using a target size (torch.Size([100])) that is different to the input size (torch.Size([500])) is deprecated. Please ensure they have the same size.